### Functional Testing

This notebook aims to test whether functional differences in the transfer characteristic can be detected through timing difference of the observation points.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from spicetestlib.base import NetList
from spicetestlib.simulator import LTSpiceSimulator
from spicetestlib.test_utilities import InjectionPoint, InverterObserver

NETLIST_PATH = os.path.join("SPICE",'amplifier_base.net')
SPICE_SIM_DATA_PATH = "SPICE_SIM_DATA"

SUPPLY_NODE = 'VDD'
GROUND_NODE = '0'
VDD = 3.3
VSS = 0

sim = LTSpiceSimulator(output_folder=SPICE_SIM_DATA_PATH)
netlist = NetList(NETLIST_PATH)

# create all possible injection points
injection_points = []
for node in netlist.net.get_all_nodes():

    # skip supply and ground nodes
    if node.lower() in [SUPPLY_NODE.lower(), GROUND_NODE.lower()]:
        continue

    # create both pull up and pull down
    injection_points.append(InjectionPoint(netlist, node, False))
    injection_points.append(InjectionPoint(netlist, node, True))

observation_points = [
    InverterObserver(netlist, 'OUT'),
    InverterObserver(netlist, 'DIF+'),
]

# add a transient analysis
netlist.net.add_instruction(".tran 200n")

# list of values for R3 to sweep
percentages = [0.9, 0.75, 0.5, 0.25, 0.1]
R3_base_value = 2e5
R3_values = []
for percentage in percentages:
    R3_values.append((1-percentage) * R3_base_value)
for percentage in reversed(percentages):
    R3_values.append((1+percentage) * R3_base_value)

# build tick labels for plots
tick_labels = []
for percentage in percentages:
    tick_labels.append(f"-{percentage*100:.0f}")
for percentage in reversed(percentages):
    tick_labels.append(f"+{percentage*100:.0f}")

# insert and activate all observation points
for obs in observation_points:
    obs.inject()
    obs.activate()
    print(str(obs))

# insert all injection points
for inj in injection_points:
    inj.inject()
    print(str(inj))

In [ ]:
results_bool = []
results_float = []

for i, inj in enumerate(injection_points):
    
    inj.activate()
    
    print(f"Running simulation for {inj}")

    results_bool.append([])
    results_float.append([])

    # set R3 to the base value
    netlist.net.set_component_value('R3', R3_base_value)

    # set expected values
    log, raw, raw_op = sim.run_now(netlist, f'expected.net')
    for obs in observation_points:
        obs.observe_expected(log, raw, raw_op)
    
    for j, R3_value in enumerate(R3_values):
        netlist.net.set_component_value('R3', R3_value)
        print(f"  Running simulation for {R3_value/1e3:.0f}k")
        result = sim.run_now_n_eval(netlist, f'sim.net', observation_points)
        results_bool[i].append(result[0])
        results_float[i].append([r[0] for r in result[1]])

    inj.deactivate()

In [ ]:
# plot result overview for all injection points

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# plotting of results
pixels_out = np.zeros(shape = (len(injection_points), len(R3_values)))
pixels_stage1 = np.zeros(shape = (len(injection_points), len(R3_values)))
for i in range(len(injection_points)):
    for j in range(len(R3_values)):
        # output observer
        if not results_bool[i][j][0]:
            pixels_out[i,j] = 1
        elif results_float[i][j][0] != 0 and abs(results_float[i][j][0]) > 1e-11:
            pixels_out[i,j] = 0.5
        # stage 1 observer
        if not results_bool[i][j][1]:
            pixels_stage1[i,j] = 1
        elif results_float[i][j][1] != 0 and abs(results_float[i][j][0]) > 1e-11:
            pixels_stage1[i,j] = 0.5

fig, ax = plt.subplots(2, 1, figsize=(5, 7))
ax[0].imshow(pixels_out, cmap='Greens', aspect='auto', extent=[0.5, len(R3_values)+0.5, len(injection_points)+0.5, 0.5])
ax[0].set_title('Output Observer')
ax[1].imshow(pixels_stage1, cmap='Greens', aspect='auto', extent=[0.5, len(R3_values)+0.5, len(injection_points)+0.5, 0.5])
ax[1].set_title('Stage 1 Observer')
for axis in ax:
    axis.set_xlabel('Deviation of R3 [%]')
    axis.set_ylabel('Injection Point')
    axis.set_xticks(np.arange(1,len(R3_values)+1,1), tick_labels)
    axis.set_xticks(np.arange(0.5, len(R3_values), 1), minor=True)
    axis.set_yticks(np.arange(0.5, len(injection_points), 1), minor=True)
    axis.grid(which='minor', color='black', linestyle='-', linewidth=0.5)

# add legend
legend_patches = [mpatches.Patch(color='#f7fcf5', label='Not Detected'),
                  mpatches.Patch(color='#73c375', label='Detectable over Timing'),
                  mpatches.Patch(color='#00441b', label='Detected')]

fig.legend(handles=legend_patches, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=3)

plt.tight_layout()
plt.savefig(os.path.join('figures', 'functional_overall.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
plt.show()
# plot only the first dataset

plt.figure(figsize=(5, 3.5))
plt.imshow(pixels_out, cmap='Greens', aspect='auto', extent=[0.5, len(R3_values)+0.5, len(injection_points)+0.5, 0.5])
plt.xlabel('Deviation of R3 [%]')
plt.ylabel('Injection Point')
plt.xticks(np.arange(1,len(R3_values)+1,1), tick_labels)
plt.xticks(np.arange(0.5, len(R3_values), 1), minor=True)
plt.yticks([1, *np.arange(4, len(injection_points)+1, 2)])
plt.yticks(np.arange(0.5, len(injection_points), 1), minor=True)
plt.grid(which='minor', color='black', linestyle='-', linewidth=0.5)

# add legend
legend_patches = [mpatches.Patch(color='#00441b', label='Detectable over Timing'),
                  #mpatches.Patch(color='#73c375', label='Detectable over Timing'),
                  mpatches.Patch(color='#f7fcf5', label='Undetectable'),]

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.legend(handles=legend_patches, loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)

plt.savefig(os.path.join('figures', 'functional_out.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
plt.title('Output Observer')
plt.show()

In [ ]:
# plot timing for injection 8

# gather timing difference data
timing_diff = []
timing_irrelev = []

for j in range(len(R3_values)):
    if not results_bool[8][j][0] or results_float[8][j][0] < -1:
        timing_irrelev.append(j)
        timing_diff.append(1e-6)
    else:
        timing_diff.append(np.abs(results_float[8][j][0]))

plt.figure(figsize=(5, 3.5))

plt.bar(range(len(timing_diff)), timing_diff, color='blue')
plt.xlabel('Deviation of R3 [%]')
plt.ylabel('Absolute Timing Difference [s]')
plt.xticks(np.arange(0,len(R3_values),1), tick_labels)
plt.xlim(-0.5, len(timing_diff)-0.5)
plt.ylim(5e-11, 5e-9)
plt.yscale('log')
for y in [1e-10, 1e-9]:
    plt.axhline(y=y, color='gray', linestyle='--', linewidth=0.5)

# add coloring and legend
for i in range(len(timing_irrelev)):
    plt.bar(timing_irrelev[i], timing_diff[timing_irrelev[i]], color='green')
legend_patches = [mpatches.Patch(color='blue', label='Detectable over Timing'),
                  mpatches.Patch(color='green', label='Detectable without Timing')]
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
#plt.legend(handles=legend_patches, loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.savefig(os.path.join('figures', 'functional_pull_down_dif+.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
plt.title('Timing Difference for Pull Down Injection @ DIF+')
plt.show()